<a href="https://colab.research.google.com/github/goin2crazy/lenta-ru-model/blob/master/lenta_ru_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model

In [9]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from torch.utils.data import DataLoader

from transformers import (AutoTokenizer,
                          AutoModelForTokenClassification,
                          AutoModelForSequenceClassification,
                          pipeline
                         )

from tqdm import tqdm

tqdm.pandas()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [26]:
class Config:
    datapath = '/kaggle/input/lenta-ru-private-dataset-for-tatar-hackathon/lenta_ru_news_2019_2023.csv'

    ner_preset = "Davlan/distilbert-base-multilingual-cased-ner-hrl"
    sentiment_preset = 'doublecringe123/lenta-ru-sentiments'

cfg = Config()

In [28]:
class NERModel:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(cfg.ner_preset)
        self.model = AutoModelForTokenClassification.from_pretrained(cfg.ner_preset).to(device)
        self.nlp = pipeline("ner", model=self.model, tokenizer=self.tokenizer, aggregation_strategy="max", device = device)

    def __call__(self, batch):
        return self.nlp(batch)

class SentimentModel:
    def __init__(self):
        model_checkpoint = cfg.sentiment_preset
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to(device)


    def get_sentiment(self, text, focus = ''):
        """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
        with torch.no_grad():
            inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
            outputs = self.model(**inputs).logits

            probas = torch.sigmoid(outputs).cpu().numpy()

        for proba in probas:
              yield proba.dot([-1, 0, 1])

    def __call__(self, *args, **kwargs):
        return self.get_sentiment( *args, **kwargs)

In [29]:
ner_model = NERModel()
sentiment_model = SentimentModel()

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.1M [00:00<?, ?B/s]

In [45]:
class Inference:
    def __init__(self):
        self.ner_model = NERModel()
        self.sentiment_model = SentimentModel()

    def predict_sentiment(self, word, text):
        """

        Args:
          word: focus word
          text: text to analize

        Returns:

        """
        p = list(self.sentiment_model(f"[focus: {word}] {text}"))

        return sum(p)/len(p)

    def add_sentiments(self, ner_text, text):
        text_sentences = text.split('.')

        ner_sentiments = {i['word']: self.predict_sentiment(i['word'], text) for i in ner_text}

        ner_sentiments = [{**nt, "sentiment": v} for nt, (k, v) in zip(ner_text, ner_sentiments.items())]
        return  ner_sentiments

    def process(self, batch):
        """
        Arguments:
            batch: list is strs
        """

        ner_batch = self.ner_model(batch)

        return [self.add_sentiments(ner_text, text) for ner_text, text in zip(ner_batch, batch)]

    def __call__(self, batch):
        if type(batch) == str:
            batch = [batch]

        return self.process(batch)

In [46]:
! python -V

Python 3.10.12
